In [11]:
from joblib_progress import joblib_progress
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import time

In [13]:
def tempFunc(i):
    pd.DataFrame([i*4])\
    .to_csv("./temp/"+str(i)+"csv")
    time.sleep(3)

with joblib_progress("Status",total = 50):
    Parallel(n_jobs=4)(delayed(tempFunc)(i) for i in range(50))

Output()

In [14]:
import netCDF4 as nc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import xarray as xr
import matplotlib.colors as pltc
import geopandas
import datetime as dt
from scipy import stats
from sklearn import preprocessing
import s3fs
import sys, os, glob,re
import multiprocessing as mp
import time as time
import fsspec
from joblib_progress import joblib_progress
from joblib import Parallel, delayed

## Cubic feet to cubic meters conversion factor
cfs_2_cms = 0.0283168466

In [15]:
pnwNP = pd.read_csv("../data/pnwNPall_InfowStats.csv")

### Pull out sites
shp = geopandas.read_file("../data/VIC_UW/shapefiles/columbia_seg.shp")
shp = pnwNP.merge(shp,how = 'left',left_on='comid',right_on='POI_ID')


In [16]:
## Open modeled datasets (VIC and PRMS) for just reach ID and runoff
pnwVIC = xr.open_mfdataset('../data/VIC_UW/vic_historical_first_route_all.nc')[['reachID','IRFroutedRunoff']]


pnwPRMS = xr.open_mfdataset('../data/VIC_UW/prms_historical_first_route_all.nc')[['reachID','IRFroutedRunoff']]

## Open NWM2.0
## Open NWM from NOAA AWS bucket
s3_path = 's3://noaa-nwm-retro-v2-zarr-pds' #nwm 2.0

# Connect to S3
s3 = s3fs.S3FileSystem(anon=True)
store = s3fs.S3Map(root=s3_path, s3=s3, check=False)

# load the dataset
ds = xr.open_zarr(store=store, consolidated=True)


## Open NWM2.1
fs = fsspec.filesystem('s3', anon=True)
_file = fs.glob('noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr')

dsd1 = xr.open_dataset(fs.get_mapper(_file[0]), engine='zarr', backend_kwargs={'consolidated': True})


In [18]:
# for i in range(len(shp)):
def getModels(i):
    print(shp['gage'][i])
    fields = ['X_00060_00003', 'Date']

    ## Get Vic data
    VIC = pnwVIC.where(pnwVIC['reachID']==shp['seg_id'][i],drop=True).to_dataframe()
    VIC = VIC.drop(['reachID'],axis=1)
    VIC= VIC.droplevel('seg')
    VIC['time'] = pd.to_datetime(VIC.index,)
    VIC['time'] = VIC['time'].dt.tz_localize(None)
    VIC = VIC.reset_index(drop=True)
    VIC.columns = ["streamflow_VIC","time"]



    ## Get PRMS data
    PRMS = pnwPRMS.where(pnwPRMS['reachID']==shp['seg_id'][i],drop=True).to_dataframe()
    PRMS = PRMS.drop(['reachID'],axis=1)
    PRMS= PRMS.droplevel('seg')
    PRMS['time'] = pd.to_datetime(PRMS.index,)
    PRMS['time'] = PRMS['time'].dt.tz_localize(None)
    PRMS = PRMS.reset_index(drop=True)
    PRMS.columns = ["streamflow_PRMS","time"]


    # ## Combine
    datMain = pd.merge(VIC,PRMS, on='time',how='outer')

    try:
        ## Get NWM 2.0 data
        # slice all data using a specific reach identifier
        dat = ds.sel(feature_id=shp['comid'][i]).streamflow.persist() 

        # This step takes a bit longer because it's actually returning the data
        dat = dat.resample(time='1d').mean()

        NWM = pd.DataFrame(dat.to_pandas())
        NWM['time'] = pd.to_datetime(NWM.index)
        NWM['time'] = NWM['time'].dt.tz_localize(None)
        NWM.columns = ["streamflow_NWM2d0","time"]
        NWM = NWM.reset_index(drop=True)


        datMain = pd.merge(datMain,NWM,on='time',how='outer')


        ## Get NWM 2.1 data
        # slice all data using a specific reach identifier
        df = dsd1.sel(feature_id=shp['comid'][i]).streamflow.persist() 

        NWM2d1 = pd.DataFrame(df.to_pandas()).resample('1d').mean()
        NWM2d1['time'] = pd.to_datetime(NWM2d1.index)
        NWM2d1['time'] = NWM2d1['time'].dt.tz_localize(None)
        NWM2d1.columns = ["streamflow_NWM2d1","time"]
        NWM2d1 = NWM2d1.reset_index(drop=True)


        # Combine with previous modeled data
        datMain = pd.merge(datMain,NWM2d1, on='time',how='outer')

        # ## Get NWIS Data
        path  = "../data/NWIS_streamflow/"
        gage = '*'+str(shp['gage'][i])+'*'
        file = glob.glob("../data/NWIS_streamflow/daily/"+gage)[0]


        fields = ['00060_Mean','datetime']
        NWIS = pd.read_csv(file,usecols=fields)
        NWIS.columns = ["time","streamflow_NWIS"]
        NWIS['time'] = pd.to_datetime(NWIS['time']).dt.tz_localize(None)
        NWIS["streamflow_NWIS"] = NWIS["streamflow_NWIS"]*cfs_2_cms

        ## Combine all the data
        datMain = pd.merge(datMain,NWIS, on='time',how='outer')
        datMain["gage"] = shp['gage'][i]


        datMain.sort_values(by='time',ascending=True) \
        .reset_index(drop=True) \
        .to_csv('../data/pnwNP_modeledData/'+str(shp['gage'][i])+".csv")

    except:
        print("No NWM data")
        pass

In [ ]:
getModels(1)

10366000
